<a href="https://colab.research.google.com/github/Droid008/Introduction-to-the-processing-of-a-natural-language/blob/Lesson-%233/HW_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!wget https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv
!wget https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv
!pip install annoy
!pip install tqdm_notebook
!pip install tqdm
!pip install pymorphy2
!pip install stop_words

--2022-06-06 12:15:01--  https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.18, 2620:100:6016:18::a27d:112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/fnpq3z4bcnoktiv/positive.csv [following]
--2022-06-06 12:15:02--  https://www.dropbox.com/s/raw/fnpq3z4bcnoktiv/positive.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc7669b40460854f6b20141de1e7.dl.dropboxusercontent.com/cd/0/inline/BmvXwzyV6iKgPDXlecCAe_z9QT-DYw6ayEDSkypVk0KUr9bhyjvazuDmtCx_FebGtPV8FNEsTJqYVNSsXiRjGYRSRCNWUdOXCl1nB_KiBJJD-kVVuklNebquF9XrVDAnj3sLhbqMu3kCE8fzAbtkDyp4iP_4w2OlHGHWnUGboaiBhw/file# [following]
--2022-06-06 12:15:02--  https://uc7669b40460854f6b20141de1e7.dl.dropboxusercontent.com/cd/0/inline/BmvXwzyV6iKgPDXlecCAe_z9QT-DYw6ayEDSkypVk0KUr9bhyjvazuDmtCx_FebGtPV8

In [4]:
import numpy as np

import pandas as pd

import string

from pymorphy2 import MorphAnalyzer

from stop_words import get_stop_words

import annoy

from gensim.models import Word2Vec, FastText

import pickle

from tqdm import tqdm_notebook

In [5]:
df_pos = pd.read_csv('positive.csv', sep=';', header=None, usecols=[3])

df_neg = pd.read_csv('negative.csv', sep=';', header=None, usecols=[3])

df = pd.concat([df_neg, df_pos])

df.columns=['text']

df

,text
0,на работе был полный пиддес :| и так каждое за...
1,"Коллеги сидят рубятся в Urban terror, а я из-з..."
2,@elina_4post как говорят обещаного три года жд...
3,"Желаю хорошего полёта и удачной посадки,я буду..."
4,"Обновил за каким-то лешим surf, теперь не рабо..."
...,...
114906,"Спала в родительском доме, на своей кровати......"
114907,RT @jebesilofyt: Эх... Мы немного решили сокра...
114908,"Что происходит со мной, когда в эфире #proacti..."
114909,"""Любимая,я подарю тебе эту звезду..."" Имя како..."


In [6]:
morpher = MorphAnalyzer()

stop_w = set(get_stop_words("ru"))

exclude = set(string.punctuation)

In [7]:
def preprocess_txt(line):
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in stop_w and i != ""]
    return spls

In [8]:
sentences = [ preprocess_txt(t) for t in tqdm_notebook(df['text']) ]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


  0%|          | 0/226834 [00:00<?, ?it/s]

In [9]:
def get_sentence_vector(model, sentence):
    r  = [model.wv[w] for w in sentence if w in model.wv ]
    
    return sum(r) / len(r) if r else np.zeros(300)

In [10]:
def build_index(model):
    index = annoy.AnnoyIndex(300 ,'angular')
    for i, snt in tqdm_notebook(enumerate(sentences)):
        v = get_sentence_vector(model, snt)
        index.add_item(i, v)
    index.build(10)
    
    return index

In [11]:
def find_nearest_tweets(model, index, tw, n=5):
    tw = preprocess_txt(tw)
    twv = get_sentence_vector(model, tw)
    nns = index.get_nns_by_vector(twv, n)
    
    return [df['text'].iloc[i] for i in nns]

In [12]:
def test_nearest_tweets(model, index, tw, n=5):
    print(50*'-')
    print(f"Nearest tweets for '{tw}':\n")
    ntws = find_nearest_tweets(model, index, tw, n)
    for tw in ntws:
        print(tw, '\n')

In [13]:
modelW2V = Word2Vec(sentences=sentences, size=300, window=5, min_count=1)

indexW2V = build_index(modelW2V)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


0it [00:00, ?it/s]

In [14]:
for i in np.random.randint(len(sentences), size=10):
    test_nearest_tweets(modelW2V, indexW2V, df['text'].iloc[i])

--------------------------------------------------
Nearest tweets for '#AutoFollow  анонсировали детский планшет  а у нас в детстве только кубики были, мама прибивала их к полу( #новости':

#AutoFollow  анонсировали детский планшет  а у нас в детстве только кубики были, мама прибивала их к полу( #новости 

#AutoFollow  анонсировали детский планшет  а у нас в детстве только кубики были, мама прибивала их к полу( #новости 

#AutoFollow  анонсировали детский планшет  а у нас в детстве только кубики были, мама прибивала их к полу( #новости 

анонсировали детский планшет а у нас в детстве только кубики были, мама прибивала их к полу( #новости 

@NikySHARP хватит о еде, ты же не такая))ох помню я одно свидание... в общем не толстей)) 

--------------------------------------------------
Nearest tweets for 'Приехали на учебу, а там света нет)) всех отпустили! Понедельник может быть хорошим!)))':

Приехали на учебу, а там света нет)) всех отпустили! Понедельник может быть хорошим!))) 

ушла вес

In [15]:
modelFT = FastText(sentences=sentences, size=300, window=5, min_count=1)

indexFT = build_index(modelFT)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


0it [00:00, ?it/s]

In [16]:
for i in np.random.randint(len(sentences), size=10):
    test_nearest_tweets(modelW2V, indexW2V, df['text'].iloc[i])

--------------------------------------------------
Nearest tweets for '@xumenen @beast_extra бля, не знал что все так плохо( Без примечаний такого уровня конечно совсем не то будет, особенно с Red Son например':

@xumenen @beast_extra бля, не знал что все так плохо( Без примечаний такого уровня конечно совсем не то будет, особенно с Red Son например 

Пиздец водить совсем разучился, и инструктор походу догадался, что правила я нихуя не знаю( 

Хотите знать правду?С Юлей разошлись из-за того,что я косяк ходячий.Бывает.Сделал массу выводов) 

@NastiaRyzhakova ахаха дада, ну он всё равно хорошенький)ахах^^ 

@HarevechYilea да черт знает... Сама в печали из-за всякой херни...((( 

--------------------------------------------------
Nearest tweets for 'На улице снег и на рабочем столе снежинки падают =)':

На улице снег и на рабочем столе снежинки падают =) 

На улице снег и на рабочем столе снежинки падают =) 

На улице снег и на рабочем столе снежинки падают =) 

На улице снег и на рабочем

In [17]:
for i in np.random.randint(len(sentences), size=10):
    test_nearest_tweets(modelFT, indexFT, df['text'].iloc[i])

--------------------------------------------------
Nearest tweets for 'вспомнила какие красивые у тебя глаза ) http://t.co/ntEKuR6HTv':

вспомнила какие красивые у тебя глаза ) http://t.co/ntEKuR6HTv 

Мужик он в любом возрасте мужик!))) http://t.co/1GsqBaYYSN 

Все по разному вспоминают меня ))хаха http://t.co/zr9fsdPBY6 

...вокруг столько девушек красивых....влюбится что ли :) http://t.co/R9trwGT1iN 

Джессика тоже любит звёздные войны! ;) http://t.co/Fu1oG6cLcR 

--------------------------------------------------
Nearest tweets for '- Вокруг красивых девушек вертятся не только парни. - Да? А что же еще? - Слухи, моя дорогая..много слухов..)))))':

- Вокруг красивых девушек вертятся не только парни. - Да? А что же еще? - Слухи, моя дорогая..много слухов..))))) 

красивые девочки жалуются на одиночество..что же тогда с некрасивыми :( 

@olya_in_da_sky наоборот согласен! а то многим, уж ооочень многим девушкам не нравятся парни с бородой((((( 

Чтобы нравиться девочкам, надо быть умны

In [18]:
qs = ['как дела', 'какая замечательная погода']

for q in qs:
    test_nearest_tweets(modelW2V, indexW2V, q, 5)

--------------------------------------------------
Nearest tweets for 'Отличное средство от':

Отличный день) благодаря тебе♥ http://t.co/b1iiU8nCmR 

Отличный стеб! Только так;) 
"Ведьмы из Сугаррамурди"
http://t.co/V7MU8l3KWK 

Отличная тарелка:) http://t.co/SPRv9rIBeA #РетвитнуТвитВзаимно 

@treeofwhales take it easy, отличный день, несмотря ни на что) 

@tugarin_alex ну не правда,не надо тут!по мне так вполне отличный день) 

--------------------------------------------------
Nearest tweets for 'Идиотская идея':

@s_l_o_t  прикол)когда будет эта идея воплощена в жизнь?) 

Ахаха..Блин..Вхаха..Мамина идея была:D http://t.co/GgXq9hKqf7 

День был хороший, прожит не зря,  но настроения нет( узнала информацию интересную мне,  но не уверена на 100% что правда. 

@RAD_AG а по идее должно быть DDDD но это Россия,детка(((((((( 

@continuumsl Я о том же,Стас.Красивое,особенное всегда доставляет удовольствие.Люблю такое.Если это вещь, то я непрем должна прикоснуться)) 



In [19]:
for q in qs:
    test_nearest_tweets(modelFT, indexFT, q, 5)

--------------------------------------------------
Nearest tweets for 'Отличное средство от':

У кого то сессичные. А у меня депрессичные ( 

сегодня весь день в хорошем настроении) только день какой-то весь нервный, люди сегодня психованные и нервные! 

моё ахиренное, классное, отличное, весёлое настроение пропало. :( 

Полное отсутствие чувства что сегодня новый год:( 

Этот новый год будет самый ужасный(
С каждым днем сильнее заболеваю( 

--------------------------------------------------
Nearest tweets for 'Идиотская идея':

Особенно Швабрин, хоть он и отрицательный герой в этой пьесе) 

Вот ни у кого нет аллергии на клюкву, а у меня есть! Особенная я))) 

Гринько с его мини-песнюшками о наших реалиях просто шедеврален!))) 

@CheOmsk ну предыдущая-то классная!! Такой трогательный момент передачи огня)) 

@kathyabodrova я как расходный материал :( 



Word2vec и Fasttext. 2-3 ответа на вопросы получились адекватные в обоих случаях, совпадение твитов - не очень, кажется, что FastText работает чуть лучше, но все все равно не достаточно. Кажется присутствует недостаточное разнообразие твитов в этом дата сете